<a href="https://colab.research.google.com/github/jo-cho/memos/blob/master/RL/sentdex_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

RL

In [0]:
!pip install -q pyvirtualdisplay
!sudo apt-get install -q xvfb

Reading package lists...
Building dependency tree...
Reading state information...
xvfb is already the newest version (2:1.19.6-1ubuntu4.3).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
'''env = gym.make("MountainCar-v0")
env.reset()
#3개의 액션

LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 25000

discrete_os_size = [20]*len(env.observation_space.high)
discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / discrete_os_size


q_table = np.random.uniform(low=-2, high=0, size=discrete_os_size + [env.action_space.n]) #20x20x3 즉 모든 combination of observation(position and velocity) 
#위는 radom q values라고 보면 됨.

def get_discrete_state(state):
  discrete_state = (state - env.observation_space.low) / discrete_os_win_size
  return tuple(discrete_state.astype(np.int))
#state들을 discrete하게, 정수 튜플로 

discrete_state = get_discrete_state(env.reset()) #initial observation
#print(discrete_state)

#print(np.argmax(q_table[discrete_state])) 
#'q value를 최대화하는 action'


done = False

while not done:
  action = np.argmax(q_table[discrete_state])
  # qvalue를 최대화하는 액션선택
  new_state, reward, done, _ = env.step(action)
  
  new_discrete_state = get_discrete_state(new_state)
  env.render()
  if not done:
    max_future_q = np.max(q_table[new_discrete_state])
    current_q = q_table[discrete_state + (action,)]
    new_q = (1-LEARNING_RATE)*current_q + LEARNING_RATE*(reward + DISCOUNT*max_future_q)
    q_table[discrete_state+(action,)] = new_q #update action that we just took
  elif new_state[0] >= env.goal_position:
    q_table[discrete_state+ (action,)] = 0

  discrete_state = new_discrete_state

env.close()'''

1000번 돌려보면서 goal position에 도달한 episode를 출력해보자

In [0]:
env = gym.make("MountainCar-v0")
env.reset()
#3개의 액션

LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 1000

show_every = 500

discrete_os_size = [20]*len(env.observation_space.high)
discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / discrete_os_size


q_table = np.random.uniform(low=-2, high=0, size=discrete_os_size + [env.action_space.n]) #20x20x3 즉 모든 combination of observation(position and velocity) 
#위는 radom q values라고 보면 됨.

def get_discrete_state(state):
  discrete_state = (state - env.observation_space.low) / discrete_os_win_size
  return tuple(discrete_state.astype(np.int))
#state들을 discrete하게, 정수 튜플로 

for episode in range(EPISODES):
  if episode % show_every == 0:
    print(episode)
    render = True
  else:
    render = False
  discrete_state = get_discrete_state(env.reset()) #initial observation
  done = False
  while not done:
    action = np.argmax(q_table[discrete_state])
    # qvalue를 최대화하는 액션선택
    new_state, reward, done, _ = env.step(action)
    new_discrete_state = get_discrete_state(new_state)
    if render:
      env.render()
    if not done:
      max_future_q = np.max(q_table[new_discrete_state])
      current_q = q_table[discrete_state + (action,)]
      new_q = (1-LEARNING_RATE)*current_q + LEARNING_RATE*(reward + DISCOUNT*max_future_q)
      q_table[discrete_state+(action,)] = new_q #update action that we just took
    elif new_state[0] >= env.goal_position:
      print(f"We made it on episode {episode}")
      q_table[discrete_state+ (action,)] = 0

    discrete_state = new_discrete_state

env.close()

0
500
We made it on episode 528
We made it on episode 532
We made it on episode 533
We made it on episode 536
We made it on episode 538
We made it on episode 554
We made it on episode 557
We made it on episode 559
We made it on episode 579
We made it on episode 581
We made it on episode 584
We made it on episode 602
We made it on episode 616
We made it on episode 682
We made it on episode 689
We made it on episode 697
We made it on episode 700
We made it on episode 701
We made it on episode 703
We made it on episode 704
We made it on episode 706
We made it on episode 707
We made it on episode 708
We made it on episode 711
We made it on episode 712
We made it on episode 713
We made it on episode 714
We made it on episode 717
We made it on episode 721
We made it on episode 722
We made it on episode 724
We made it on episode 727
We made it on episode 735
We made it on episode 817
We made it on episode 851
We made it on episode 890
We made it on episode 899
We made it on episode 913
We mad

엡실론을 넣자 (exploration개념)

In [0]:
'''env = gym.make("MountainCar-v0")
env.reset()
#3개의 액션

LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 1000

show_every = 500

discrete_os_size = [20]*len(env.observation_space.high)
discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / discrete_os_size

# Exploration settings
epsilon = 1          # not a constant, qoing to be decayed
start_epsilon_decaying = 1
end_epsilon_decaying = EPISODES // 2

epsilon_decay_value = epsilon/(end_epsilon_decaying - start_epsilon_decaying)

q_table = np.random.uniform(low=-2, high=0, size=discrete_os_size + [env.action_space.n]) #20x20x3 즉 모든 combination of observation(position and velocity) 
#위는 radom q values라고 보면 됨.

def get_discrete_state(state):
  discrete_state = (state - env.observation_space.low) / discrete_os_win_size
  return tuple(discrete_state.astype(np.int))
#state들을 discrete하게, 정수 튜플로 

for episode in range(EPISODES):
  if episode % show_every == 0:
    print(episode)
    render = True
  else:
    render = False
  discrete_state = get_discrete_state(env.reset()) #initial observation
  done = False
  while not done:
    action = np.argmax(q_table[discrete_state])
    # qvalue를 최대화하는 액션선택
    new_state, reward, done, _ = env.step(action)
    new_discrete_state = get_discrete_state(new_state)
    if render:
      env.render()
    if not done:
      max_future_q = np.max(q_table[new_discrete_state])
      current_q = q_table[discrete_state + (action,)]
      new_q = (1-LEARNING_RATE)*current_q + LEARNING_RATE*(reward + DISCOUNT*max_future_q)
      q_table[discrete_state+(action,)] = new_q #update action that we just took
    elif new_state[0] >= env.goal_position:
      print(f"We made it on episode {episode}")
      q_table[discrete_state+ (action,)] = 0

    discrete_state = new_discrete_state

  if end_epsilon_decaying >= episode >= start_epsilon_decaying:
    epsilon -= epsilon_decay_value #줄어드는 엡실론

env.close()'''

cloned code

In [0]:
# objective is to get the cart to the flag.
# for now, let's just move randomly:

import gym
import numpy as np

env = gym.make("MountainCar-v0")

LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 2000
SHOW_EVERY = 500

DISCRETE_OS_SIZE = [20, 20]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE

# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)


q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))


def get_discrete_state(state):
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size
    return tuple(discrete_state.astype(np.int))  # we use this tuple to look up the 3 Q values for the available actions in the q-table


for episode in range(EPISODES):
    discrete_state = get_discrete_state(env.reset())
    done = False

    if episode % SHOW_EVERY == 0:
        render = True
        print(episode)
    else:
        render = False

    while not done:

        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(q_table[discrete_state])
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.n)


        new_state, reward, done, _ = env.step(action)

        new_discrete_state = get_discrete_state(new_state)

        if episode % SHOW_EVERY == 0:
            env.render()
        #new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

        # If simulation did not end yet after last step - update Q table
        if not done:

            # Maximum possible Q value in next step (for new state)
            max_future_q = np.max(q_table[new_discrete_state])

            # Current Q value (for current state and performed action)
            current_q = q_table[discrete_state + (action,)]

            # And here's our equation for a new Q value for current state and action
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            # Update Q table with new Q value
            q_table[discrete_state + (action,)] = new_q


        # Simulation ended (for any reson) - if goal position is achived - update Q value with reward directly
        elif new_state[0] >= env.goal_position:
            print(f"We made it on episode {episode}")
            #q_table[discrete_state + (action,)] = reward
            q_table[discrete_state + (action,)] = 0

        discrete_state = new_discrete_state

    # Decaying is being done every episode if episode number is within decaying range
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value


env.close()

0
500
We made it on episode 844
We made it on episode 928
We made it on episode 941
We made it on episode 994
1000
We made it on episode 1009
We made it on episode 1010
We made it on episode 1016
We made it on episode 1027
We made it on episode 1030
We made it on episode 1035
We made it on episode 1038
We made it on episode 1042
We made it on episode 1050
We made it on episode 1074
We made it on episode 1075
We made it on episode 1076
We made it on episode 1077
We made it on episode 1078
We made it on episode 1079
We made it on episode 1082
We made it on episode 1084
We made it on episode 1085
We made it on episode 1086
We made it on episode 1087
We made it on episode 1089
We made it on episode 1090
We made it on episode 1091
We made it on episode 1092
We made it on episode 1093
We made it on episode 1129
We made it on episode 1136
We made it on episode 1137
We made it on episode 1141
We made it on episode 1142
We made it on episode 1143
We made it on episode 1144
We made it on episode

출처:

https://youtu.be/Gq1Azv_B4-4?list=PLQVvvaa0QuDezJFIOU5wDdfy4e9vdnx-7


https://pythonprogramming.net/q-learning-algorithm-reinforcement-learning-python-tutorial/